# Code Optimization

Code optimization is the process of making an application work more efficiently, usually without modifying its functionality and accuracy. Code optimization is usually concerned with the speed of processing, but can also be used to minimize the usage of different resources, such as memory, disk space, or network bandwidth.

Jaworski , Michal and Tarek Ziadé. Expert Python Programming. Packt Publishing, 2019.


Even the best logging, metrics, and tracing systems will give you only a rough overview of the performance problem. If you decide to fix it, you will have to perform a careful profiling process that will uncover detailed resource usage patterns

What are the main performance killers?
- Excessive complexity
- Excessive resource allocation and resource leaks
- Excessive I/O and blocking operations


## Code complexity
The two most popular ways to define application complexity are as follows:

**Cyclomatic complexity** - which is very often correlated with application performance.

**The Landau notation** - also known as big O notation, is an algorithm classification method that is useful in objectively judging code performance.

## Cyclomatic complexity
Short and sweet higher complexity = lower performance

| | | 
|:---|:---|
|Cyclomatic complexity value|Complexity class|
|1 to 10|Not complex|
|11 to 20|Moderately complex|
|21 to 50|Really complex|
|Above 50|Too complex|

## The big O notation
Defines how an algorithm is affected by the size of the input
To measure the big O notation, all constants and low-order terms are removed in order to focus on the portion that really matters when the size of the input data grows very large.


This is commonly expressed using **Big O notation**.

| Big O        | Name                | Example                |
|--------------|---------------------|------------------------|
| O(1)         | Constant time        | Accessing an array element |
| O(log n)     | Logarithmic time     | Binary search           |
| O(n)         | Linear time          | Iterating over a list   |
| O(n log n)   | Linearithmic time    | Efficient sorting (e.g., mergesort) |
| O(n²)        | Quadratic time       | Nested loops over data  |
| O(2ⁿ), O(n!) | Exponential / factorial | Recursive combinatorics |

### Example
```python
def function(n):
    for i in range(n):
        print(i)

```

the print() function will be executed n times therefor O(n)

## Profiling CPU usage

There are two ways to profile the code:

**Macro-profiling** - This profiles the whole program while it is being used and generates statistics.

**Micro-profiling** - This measures a precise part of the program by instrumenting it manually.




## Macro-profiling
You can do different tools tow of the available to you in Python:

- `profile`: A pure-Python profiler suitable for teaching or light use.
- `cProfile`: A C-optimized profiler, more efficient and widely used in practice.

In [26]:
import time
class runIT(object):
    def __init__(self):
        for i in range(5): 
            self.heavy() 

    def medium(self): 
        time.sleep(0.01) 
     
    def light(self): 
        time.sleep(0.001) 
     
    def heavy(self): 
        for i in range(100): 
            self.light() 
            self.medium() 
            self.medium() 
        time.sleep(2) 

In [27]:
%%prun -s cumulative -q -l 10 -T prun0
runs = runIT()

 
*** Profile printout saved to text file 'prun0'.


In [28]:
print(open('prun0', 'r').read())

         3814 function calls (3798 primitive calls) in 20.614 seconds

   Ordered by: cumulative time
   List reduced from 169 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   20.613   20.613 <string>:1(<module>)
        1    0.000    0.000   20.613   20.613 4277315889.py:3(__init__)
     1505   20.594    0.014   20.594    0.014 {built-in method time.sleep}
        5    0.004    0.001   18.447    3.689 4277315889.py:13(heavy)
     1000    0.002    0.000   10.065    0.010 4277315889.py:7(medium)
        7    0.001    0.000    2.164    0.309 base_events.py:1910(_run_once)
      500    0.001    0.000    0.532    0.001 4277315889.py:10(light)
      7/5    0.000    0.000    0.008    0.002 selectors.py:451(select)
        1    0.000    0.000    0.004    0.004 decorator.py:229(fun)
        1    0.000    0.000    0.004    0.004 history.py:54(only_when_enabled)


The meaning of each column is as follows:


| Column        | Description                                                                 |
|---------------|-----------------------------------------------------------------------------|
| `ncalls`      | Total number of calls to the function                                        |
| `tottime`     | Total time spent in the function (excluding subcalls)                        |
| `percall`     | `tottime` divided by `ncalls` (avg time per direct call)                    |
| `cumtime`     | Cumulative time including all subcalls                                      |
| `percall`     | `cumtime` divided by `ncalls` (avg time including subcalls)                 |


In [29]:
import cProfile
profiler = cProfile.Profile()
profiler.runcall(runIT)
profiler.print_stats()

         3696 function calls (3688 primitive calls) in 20.621 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      500    0.001    0.000    0.533    0.001 4277315889.py:10(light)
      6/5    0.004    0.001   17.890    3.578 4277315889.py:13(heavy)
      2/1    0.000    0.000   13.952   13.952 4277315889.py:3(__init__)
     1000    0.003    0.000   10.078    0.010 4277315889.py:7(medium)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 asyncio.py:200(_handle_events)
        1    0.000    0.000    0.000    0.000 asyncio.py:225(add_callback)
        4    0.000    0.000    0.000    0.000 attrsettr.py:42(__getattr__)
        4    0.000    0.000    0.000    0.000 attrsettr.py:65(_get_attr_opt)
        1    0.000    0.000    0.000    0.000 base_events.py:1895(_add_callback)
        2    0.000    0.000    0.000    0.000 base_events.py:

## Profiling your code line-by-line with line_profiler

Sometimes we need an even more detailed analysis of code performance 

This is particularly useful when optimizing algorithms, loops, or numerically intensive routines.

To profile code line-by-line, we need an external Python module named `line_profiler`.


```bash
pip install line_profiler
```


In [30]:
%%writefile runIT.py 
import time
def medium(): 
    time.sleep(0.01) 
 
def light(): 
    time.sleep(0.001) 
 
def heavy(): 
    for i in range(100): 
        light() 
        medium() 
        medium() 
    time.sleep(2) 
 
def runit(n): 
    for i in range(n): 
        heavy()

Overwriting runIT.py


In [31]:
from runIT import runit

In [32]:
import numpy as np
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


The `line_profiler` package integrates with IPython via the `%lprun`

In [33]:
#%lprun -T lprof0 -f simulate simulate(50)
%lprun -T lprof0 -f runit runit(5)


*** Profile printout saved to text file 'lprof0'. 


Timer unit: 1e-09 s

Total time: 20.6245 s
File: /media/krispy/4A94A7D594A7C235/Users/Power/Documents/ETE/Git/ETE4990/LectureNotes/Lecture 18 - Optimization/runIT.py
Function: runit at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
    15                                           def runit(n): 
    16         6      13454.0   2242.3      0.0      for i in range(n): 
    17         5        2e+10    4e+09    100.0          heavy()

In [ ]:
print(open('lprof0', 'r').read())

In [ ]:
#%%writefile rates.py
#from condenced from Lecture 11
import threading
import requests
import json
def fetch_rate(bases, symbols =['eur','jpy','usd'] ):
    for base in bases:
        web = "http://www.floatrates.com/daily/"+str(base)+".json"
        response = requests.get(web)
        rate = response.json()
        rate[base]= {'rate':1}
        
        #create a line to output the rate
        rates_line = ", ".join(
            [f"{symbol}{float(rate[symbol]['rate']):10.04}" 
             for symbol in symbols]
        )
        print(f"{base} = {rates_line}")

In [ ]:
%lprun -T lprof1 -f fetch_rate fetch_rate(['eur','jpy','usd','rub','cad'])

In [ ]:
#to profile multiple function within the code you can do the following

from line_profiler import LineProfiler
lp = LineProfiler()
lp.add_function(light)
lp.add_function(medium)
lp.add_function(heavy)
lpp = lp(runit)
lpp(1)


lp.print_stats()


In [ ]:
print(open('lprof2', 'r').read())

# Profiling the Memory Usage

Memory profiling helps identify unnecessary allocations that can degrade long-running systems.

Writing memory-efficient code is critical for performance, especially in high-throughput or data-intensive applications (e.g., when working with large NumPy arrays or data frames).


The `memory_profiler` package integrates with IPython via the `%memit` magic command, allowing you to measure the memory usage of individual lines or functions.


``` bash
!pip install memory_profiler
````

In [ ]:
%load_ext memory_profiler
from rates import fetch_rate

In [ ]:
%mprun -T mprof0 -f fetch_rate fetch_rate(['eur','jpy','usd','rub','cad'])

In [ ]:
%mprun -T mprof1 -f runit runit(5)

In [ ]:
%mprun -T mprof2 -f my_func my_func()

the memory_profiler IPython extension also comes with a %memit magic command that lets us benchmark the memory used by a single Python statement

In [ ]:
%%memit 
with open('ItemData.json') as f:
    items = json.load(f)


# Practical Speed Improvements

In [ ]:
import random
l = [random.normalvariate(0,1) for i in range(100000)]
print(type(l))

In [ ]:
#function that computes the sum of all numbers in that list
def sum1():
    res = 0
    for i in range(len(l)):
        res = res + l[i]
    return res
a = %timeit sum1()

In [ ]:
#same function but using the fact that python can enumerate 
#the elements of a list using for x in l instead of iterating with an index
def sum2():
    res = 0
    for x in l:
        res = res + x
    return res
b = %timeit sum2()

In [ ]:
#using pythongs built-in function to 
#compute the sum of all elements in a list
def sum3():
    return sum(l)
c = %timeit sum3()

Strings

In [ ]:
strings = ['%.3f' % x for x in l]

In [ ]:
#A function concatenating all strings in that list
def concat1():
    cat = strings[0]
    for s in strings[1:]:
        cat = cat + ', ' + s
    return cat
%timeit concat1()

In [ ]:
#A function using pythongs built in string concatinator
def concat2():
    return ', '.join(strings)

%timeit concat2()

In [ ]:
print(concat1()[:24])
print(concat2()[:24])

In [ ]:
print(strings[:3])
print(l[:3])

I hope this shows you the power of generators and functional programs. 

# OMG ITS SO FAST

[Numba](https://numba.pydata.org/) is a Just-in-Time (JIT) compiler that translates a subset of Python (primarily numerical code) into optimized machine code using LLVM.


Performance speedups when compared to pure Python code can reach several orders of magnitude (10x to 1000x) and may even outmatch manually-vectorized NumPy code.

In this section, we will show you how to accelerate pure Python code generating a Mandelbrot fractal.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
size = 400
iterations = 100

To enable JIT acceleration, decorate your functions with:

```python
from numba import jit

@jit
def compute(...):
    #code goes here
```


In [ ]:
@jit
def mandelbrot_python(size, iterations):
    m = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            c = (-2 + 3. / size * j +
                 1j * (1.5 - 3. / size * i))
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z * z + c
                    m[i, j] = n
                else:
                    break
    return m

In [ ]:
m = mandelbrot_python(size, iterations)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(np.log(m), cmap=plt.cm.hot)
ax.set_axis_off()

In [ ]:
%timeit mandelbrot_python(size, iterations)

In [ ]:
#!pip install numba
from numba import jit

In [ ]:
%timeit mandelbrot_python(size, iterations) #do it again with numba

## NOTE on JIT Compilation: 
Python bytecode is normally interpreted at runtime by the Python interpreter (most often, CPython). 

Numba functions are parsed and translated directly to machine code ahead of execution, using a powerful compiler architecture named Low Level Virtual Machine (LLVM).

Numba generally gives the most impressive speedups on functions that involve tight loops on NumPy arrays.



The @jit can only compile code that is also decorated with @jit or there is a known replacement inside Numba.

Please see http://numba.pydata.org/numba-doc/latest/user/5minguide.html#will-numba-work-for-my-code for a quick description of what Numba support.